In [18]:
import pandas as pd
import numpy as np
import re

In [19]:
# Loading data
dfCSV = pd.read_csv('last3Months.csv', encoding='utf-8')

In [20]:
# Cutting specific hours from the dates
datesList = list(dfCSV['Publish Date'])
for index in range(len(datesList)):
    datesList[index] = datesList[index][:10]

In [21]:
# Getting tags list
tagsList = list(dfCSV['Flair'])

In [22]:
# Building DataFrame for counting tags & all comissions
dfDatesTags = pd.DataFrame(list(zip(datesList, tagsList)), columns=['Date', 'Tag'])

In [23]:
groupedTotal = dfDatesTags.groupby(['Date']).size()
# Getting every unique date
dayDates = list(groupedTotal.keys())
# Getting total number of submissions per day
dayTotal = list(groupedTotal.values)

In [24]:
groupedTags = dfDatesTags.groupby(['Date', 'Tag']).size()
# Dictionary with unique tags
tagsDictionary = {}
# 2-dimensional list of keys
keys2D = groupedTags.keys()

# Loop over every (date, tag) touple
for key in keys2D:
    # getting a tag
    tag = key[1]
    if tag not in tagsDictionary:
        tagsDictionary[tag] = []

In [25]:
# For every day
for day in dayDates:
    # For every tag
    for tag in tagsDictionary:
        # Append to a list number of times the tag appeared
        try:
            tagsDictionary[tag].append(groupedTags[day][tag])
        except KeyError:
            tagsDictionary[tag].append(0)

In [26]:
# Adding previously computed values to the dictionary
tagsDictionary['Date'] = dayDates
tagsDictionary['Total'] = dayTotal

In [27]:
# Building final dataframe
dfFinal = pd.DataFrame.from_dict(tagsDictionary)

In [28]:
# Dropping last column (not a full - 24h day)
dfFinal.drop(dfFinal.tail(1).index, inplace=True)

In [29]:
dfFinal.tail(5)

,Behind Soft Paywall,Hong Kong,Israel/Palestine,No ALL CAPS,No Redirect URLs,No Twitter,No videos,North Korea,Not Appropriate Subreddit,Opinion/Analysis,...,No Petitions,Not A News Article (PDF),No memes,No images,Auto-translated,COVID-19,No 'x-post' in title,Very Out Of Date,Date,Total
85,23,4,1,3,9,6,34,4,3,9,...,0,2,0,0,0,434,0,0,2020-03-20,1498
86,30,3,3,0,7,4,30,8,4,6,...,1,1,0,0,0,363,0,0,2020-03-21,1277
87,21,1,1,3,4,5,31,0,3,3,...,0,0,0,1,0,375,0,0,2020-03-22,1328
88,26,5,3,4,13,3,21,0,5,2,...,0,1,0,1,1,433,0,0,2020-03-23,1498
89,36,3,2,5,8,11,27,1,0,5,...,0,2,1,0,1,580,0,0,2020-03-24,1806


In [30]:
# COVID-19 tag was introduced in March, let's see how many non-tagged entries there are
import re
dfNotTagged = dfCSV.loc[(dfCSV['Title'].str.contains('virus|sars|covid|pandemic|epidemy', flags=re.I, regex=True)) & (dfCSV['Flair'].isna())]

In [31]:
# Cutting hours from the dates
notTaggedDates = list(dfNotTagged['Publish Date'])
for index in range(len(notTaggedDates)):
    notTaggedDates[index] = notTaggedDates[index][:10]

In [32]:
dfNotTagged['Publish Date'] = notTaggedDates
dfNotTaggedGroup = dfNotTagged.groupby(['Publish Date']).size()

In [33]:
dfNotTaggedGroup = dfNotTaggedGroup.reset_index()
dfNotTaggedGroup.rename(columns={'Publish Date': 'Date', 0: 'COVID-not-tagged'}, inplace=True)

In [34]:
# Left-join dfFinal with dfNotTaggedGroup
dfFinalMerged = dfFinal.merge(dfNotTaggedGroup, on='Date', how='left')

In [35]:
# Filling NaN with 0's
dfFinalMerged['COVID-not-tagged'].fillna(0, inplace=True)
# Changing column's data type to int (previously NaN required float)
dfFinalMerged = dfFinalMerged.astype({'COVID-not-tagged': int})

In [37]:
dfFinalMerged.head(10)

,Behind Soft Paywall,Hong Kong,Israel/Palestine,No ALL CAPS,No Redirect URLs,No Twitter,No videos,North Korea,Not Appropriate Subreddit,Opinion/Analysis,...,Not A News Article (PDF),No memes,No images,Auto-translated,COVID-19,No 'x-post' in title,Very Out Of Date,Date,Total,COVID-not-tagged
0,5,6,8,2,1,1,4,2,5,2,...,0,0,0,0,0,0,0,2019-12-26,441,0
1,18,4,12,1,3,1,6,5,11,2,...,0,0,0,0,0,0,0,2019-12-27,741,0
2,21,7,4,0,1,2,13,4,2,3,...,0,0,0,0,0,0,0,2019-12-28,637,0
3,14,7,4,0,0,0,7,7,0,2,...,0,0,0,0,0,0,0,2019-12-29,626,0
4,14,6,6,0,0,2,14,10,0,4,...,0,0,0,0,0,0,0,2019-12-30,701,1
5,10,12,5,1,1,3,15,4,1,0,...,0,0,0,0,0,0,0,2019-12-31,691,1
6,11,13,6,0,0,3,6,4,0,2,...,0,0,0,0,0,0,0,2020-01-01,531,3
7,11,13,15,0,1,4,3,2,11,7,...,0,0,0,0,0,0,0,2020-01-02,723,3
8,16,13,6,1,20,29,18,2,23,61,...,0,0,0,0,0,0,0,2020-01-03,1065,4
9,7,11,3,4,9,13,16,1,22,39,...,0,0,0,0,0,0,0,2020-01-04,891,1


Now it is a good idea to get rid of trashy posts (tagged as 'No ALL CAPS', 'No vides', 'No Redirect URLs'....)

In [38]:
# It will be faster to specify which tags to keep
tagsToKeep = ['Hong Kong', 'Israel/Palestine', 'North Korea', 'Russia', 'Trump', 'COVID-19']
# We also need those
tagsToKeep = tagsToKeep + ['Date', 'Total', 'COVID-not-tagged']

In [39]:
# Drop all trashy columns
for column in dfFinalMerged.columns:
    if column not in tagsToKeep:
        # Subtract deleted columns from the total count
        dfFinalMerged['Total'] = dfFinalMerged['Total'] - dfFinalMerged[column]
        # Delete the column
        dfFinalMerged.drop(column, axis=1, inplace=True)

In [40]:
# Info about total # COVID posts (tagged + not-tagged)
totalCovid = list(dfFinalMerged['COVID-not-tagged'] + dfFinalMerged['COVID-19'])
dfFinalMerged['Total COVID'] = totalCovid

In [41]:
dfFinalMerged.head(20)

,Hong Kong,Israel/Palestine,North Korea,Russia,Trump,COVID-19,Date,Total,COVID-not-tagged,Total COVID
0,6,8,2,15,15,0,2019-12-26,421,0,0
1,4,12,5,19,7,0,2019-12-27,692,0,0
2,7,4,4,17,13,0,2019-12-28,583,0,0
3,7,4,7,8,10,0,2019-12-29,600,0,0
4,6,6,10,16,25,0,2019-12-30,656,1,1
5,12,5,4,14,23,0,2019-12-31,653,1,1
6,13,6,4,3,11,0,2020-01-01,504,3,3
7,13,15,2,4,11,0,2020-01-02,640,3,3
8,13,6,2,13,38,0,2020-01-03,665,4,4
9,11,3,1,9,38,0,2020-01-04,619,1,1
